# 1 of 2 Oblivious Transfer (OT)

In a oblivious transfer protocol, a sender wants to send a message $m_0$ or $m_1$ to a receiver. The sender wants the receiver to see only one message and the receiver whants the sender not to know which message he picked. To have a more in deep introduction to oblivious transfer go [here](https://en.wikipedia.org/wiki/Oblivious_transfer).
 
First The sender generates the public and private RSA keys

In [1]:
from crypt import RSAKeyGenerator
from random import randrange, seed

seed(1)
bits = 16
PublicKey, PrivateKey = RSAKeyGenerator(bits)

print(f"Public Key is (N, e)=({PublicKey[0]}, {PublicKey[1]})")
print(f"Private Key is (N, d)=({PrivateKey[0]}, {PrivateKey[1]})")

assert(PublicKey[0]==PrivateKey[0])
N = PublicKey[0]

Public Key is (N, e)=(3622687927, 411983603)
Private Key is (N, d)=(3622687927, 71648603)


Now the sender picks two mesages, for instance $m_0$ and $m_1$. In this example I'm going to generate them randomly but in principle should be two meaningful messages that the sender wants to transmit. He wants to send one and only one to the receiver but without knowing which one the receiver picked.

In [2]:
m = [randrange(2,N) , randrange(2, N)]

print(f"The messages are:\nm_0={m[0]}\nm_1={m[1]}")

The messages are:
m_0=3365773597
m_1=3523960155


And he publishes two random messages $x_0$ and $x_1$

In [3]:
x = [randrange(2, N), randrange(2, N)]
print("The sender publishes two random messages:\nx0={}\nx1={}".format(x[0], x[1]))

The sender publishes two random messages:
x0=3180437945
x1=2201170458


The receiver secrelty flips a coin $b$ that determines which message is going to pick, either 0 or 1. Also generates a random number in between 2 and N and keeps this two values private.

In [4]:
b = randrange(0, 2)
k = randrange(2, N)
print("b={}\nk={}".format(b, k))

b=0
k=2227706267


The receiver computes $v$, this is, $$v = (x_b + k^e)modN$$ and sends this value to the sender

In [5]:
from crypt import fastPowering

e = PublicKey[1]
v = (x[b] + fastPowering(k, e, N))%N

print("v={}".format(v))

v=2412057012


The sender now tries to revert $v$ to get $k$,

$$k_0 = (v - x_0)^d modN$$
$$k_1 = (v - x_1)^d modN$$

Either $k_0$ or $k_1$ is the value of $k$ hat the receiver picked

In [6]:
d = PrivateKey[1]
kl = [pow(v-x[0], d, N), pow(v-x[1], d, N)]

print("k0={}\nk1={}".format(kl[0], kl[1]))

k0=2227706267
k1=1724681342


The sender sends the new masked messages $$m_0^\prime = m_0 + k_0$$ $$m_1^\prime = m_1 + k_1$$ to the receiver

In [7]:
mp = [(m[0]+kl[0]), (m[1]+kl[1])]

print("m0p={}\nm1p={}".format(mp[0], mp[1]))

m0p=5593479864
m1p=5248641497


Then the receiver decrypts the message using his private key $k$.

In [8]:
mb = (mp[b] - k)

print("The message that the receiver decrypted is {}".format(mb))

The message that the receiver decrypted is 3365773597
